## Ensemple Learning

- ensemble learning is an approach to machine learning which tries to obtain better predictive performance by combining the predictions from multiple machine learning models.
- the 3 main classes of ensemble learning include:
  1. **bagging**
  2. **stacking**
  3. **boosting**

- ***Bagging***:- involves fitting many decision trees on different samples og the same data set and then averaging the results in predictions.
- ***stacking***:- fit many different model types on the same data and use another model to learn how to best combine the predictions.
- ***boosting***:- involves adding ensemble members sequentially that correct the predictions made by prior models and outputs of waiting average, often predictions.

 the above are aggregate techniques built to reduce variance. 

## Random forest: rule-based Algorithm

- Decision trees, though known as simple algorithm they are also referred to greedy algorithm. this is because they focus on optimizing for the node splits at hand, rather than taking into account how the split impacts the entire tree often resulting into overfitting. An overfit tree is highly optimized in predicting the values in the training data set which results into a learning model with a *high variance*.

**random forests and overfitting**

- overfitting is when a model is nolonger as accurate as we want it to be on thr data. 



